In [ ]:
%env MONGODB_VERSION      4.0.11
%env MONGODB_PATH mongodb-4.0.11

In [ ]:
!pip3 install pymongo

In [ ]:
!${MONGODB_PATH}/bin/mongo --help

In [ ]:
!${MONGODB_PATH}/bin/mongo --eval "printjson(db.serverStatus())"

In [ ]:
!${MONGODB_PATH}/bin/mongoimport --db mkt --collection customers --drop --jsonArray --file dataset/olist_customers_dataset.json

In [ ]:
!${MONGODB_PATH}/bin/mongoimport --db mkt --collection payments --drop --jsonArray --file dataset/olist_order_payments_dataset.json

In [ ]:
!${MONGODB_PATH}/bin/mongoimport --db mkt --collection orders --drop --jsonArray --file dataset/olist_orders_dataset.json

In [ ]:
!${MONGODB_PATH}/bin/mongoimport --db mkt --collection products --drop --jsonArray --file dataset/olist_products_dataset.json

In [ ]:
!${MONGODB_PATH}/bin/mongoimport --db mkt --collection sellers --drop --jsonArray --file dataset/olist_sellers_dataset.json

In [ ]:
!${MONGODB_PATH}/bin/mongoimport --db mkt --collection productcategory --drop --jsonArray --file dataset/product_category_name_translation.json

In [ ]:
!${MONGODB_PATH}/bin/mongoimport --db mkt --collection orderitems --drop --jsonArray --file dataset/olist_order_items_dataset.json

In [ ]:
!${MONGODB_PATH}/bin/mongo --quiet --eval 'db.sellers.findOne();' mkt

In [ ]:
from pymongo import MongoClient
import json

cliente = MongoClient('localhost', 27017)

### Order id: 00010242fe8c5a6d1ba2dd792cb16214
ordersItems = cliente.mkt.orderitems
products = cliente.mkt.products

resultOrderItems = ordersItems.find({"order_id": "00010242fe8c5a6d1ba2dd792cb16214"})
orderProducts = []
for orderItem in resultOrderItems:
    resultProduct = products.find({"product_id": orderItem["product_id"]}, {"_id": 0})
    orderProducts.append(resultProduct[0])
    
print(json.dumps(orderProducts, indent=4, sort_keys=True))

In [ ]:
from pymongo import MongoClient
import json

cliente = MongoClient('localhost', 27017)

ordersItems = cliente.mkt.orderitems
products = cliente.mkt.products

resultOrderItems = ordersItems.aggregate([
    {"$group": {"_id": "$product_id", "count": {"$sum": 1}}},
    {"$sort": { "count": -1 } },
    {"$limit": 10}
])
    
orderProducts = []
for orderItem in resultOrderItems:
    resultProduct = products.find({"product_id": orderItem["_id"]}, {"_id": 0})
    resultProduct = {"count": orderItem["count"], "product": resultProduct[0]}
    orderProducts.append(resultProduct)
    
print(json.dumps(orderProducts, indent=4, sort_keys=True))

In [ ]:
from pymongo import MongoClient
import json
import pprint

pp = pprint.PrettyPrinter(indent=4)
cliente  = MongoClient('localhost', 27017)
payments = cliente.mkt.payments

resultPayments = payments.aggregate([{"$group" : {'_id':{'payment_type':"$payment_type"}, 'count':{'$sum':1}}},{'$sort':{"count":-1}}])

pp.pprint(list(resultPayments))